In [ ]:
import streamlit as st
import jsonlines
import random
import pandas as pd
import json
import fileinput
import numpy as np
from pprint import pprint

In [ ]:
#Importing Dataset
olafScholzJsonLines = jsonlines.open("OlafScholz.jl")
olafScholzTwitter = pd.read_json("OlafScholz.jl", lines = True)

#First view of given Dataset
print("Datensatzlänge: " + str(olafScholzTwitter.info()))
print(olafScholzTwitter.head())

In [ ]:
#Analyzin Dictionarys
print("Response Metadaten: " + str(olafScholzTwitter["response"][0].keys()))
ResponseKeys = olafScholzTwitter["response"][0].keys()
print(str(ResponseKeys)+"\n")
for Key in ResponseKeys:
    print("Type of of Response Key {}: ".format(Key) + str(type(olafScholzTwitter["response"][0][Key])))
    #print("{}-Metadaten: ".format(Key) + str(olafScholzTwitter["response"][0][Key]))

In [ ]:
#print(olafScholzTwitter["response"][0].len)
#print("---")
#Starting to investigate Dataset
#print(olafScholzTwitter.describe())
#print(type(olafScholzTwitter))
#print(olafScholzTwitter.keys())
#for key in olafScholzTwitter:
#    print("{} : {}".format(key, olafScholzTwitter[key]))
#    print("{}".format(key) + str(type(olafScholzTwitter[key])))
    

In [ ]:
#Block for testing Dataset and figuring out Key parameters, comment out before finaly running to get propper result
"""
for line in olafScholzJsonLines:
    #print(line)
    keyResponse = line["response"]
    print("Response-Type:" + str(type(keyResponse)))
    print("Response-Keys:" + str(keyResponse.keys()))
    
    for key in keyResponse:
        print(type(keyResponse[key]))
        #pprint(keyResponse[key])
        #print("{}-Metadaten: ".format(key) + str(keyResponse[key].keys()))
    print("Includes-Metadaten: " + str(keyResponse["includes"].keys())+ "\n")
 
    print("Includes-Keys:" + str(keyResponse["includes"]["users"][0].keys())+"\n")
    #pprint("Example Includes:" +"\n" + str(keyResponse["includes"]["users"])+"\n")
        
    print("Meta-Metadaten: " + str(keyResponse["meta"].keys())+ "\n")
    data = keyResponse["data"]
    print("Type-Data: " + str(type(data))+ "\n")
    print(str(data[0]) + "\n")
    print("Type-Data[0]: " + str(type(data[0])) + "\n")
    print("data[0].keys(): " + str(data[0].keys()) + "\n")
    #print(dir(data[0].keys()))
    print("len(data[0].keys()): " + str(len(data[0].keys())) + "\n")
    break
"""

In [ ]:
# Reply Dataframe
# Importing Data into final Dataframe for ML
replyDataFrame = pd.DataFrame(columns = ["target", "ids", "date", "flag", "user", "text"])
print(replyDataFrame.head())
#dir(replyDataFrame)
iterator = 0 #Debugging Counter
for line in olafScholzJsonLines:
    keyResponse = line["response"]
    data = keyResponse["data"]
    #print("Iterator {} has length: ".format(iterator) + str(len(data)))
    #pprint(data)
    #["target", "ids", "date", "flag", "user", "text"]
    newDataRow = [None, None, None, None, None, None]
    #print(line.keys())
    userName = line["account_name"]
    #print("User Name: " + str(line["account_name"])+"\n")
    newDataRow[4] = userName
    keyResponse = line["response"]
    data = keyResponse["data"]


    for tweet in data:
        #print("Tweet-ID: " + tweet["id"])
        #print(str(tweet) + "\n")
        tweetTarget = None
        tweetID = tweet["id"]
        authorID = tweet["author_id"]
        tweetDate = tweet["created_at"][0:10]
        tweetFlag = "NO_QUERY"
        tweetText = tweet["text"]
        #tweetAuthor = tweet["context_annotations"]
        #tweetAuthor = tweet["context_annotations"]
        #tweetAuthor = userName
        #print(type(tweetAuthor))
        
        newDataRow[0] = tweetTarget
        newDataRow[1] = tweetID
        newDataRow[2] = tweetDate
        newDataRow[3] = tweetFlag
        #newDataRow[4] = tweetAuthor
        newDataRow[5] = tweetText
        #pprint(newDataRow)
        replyDataFrame.loc[len(replyDataFrame)] = newDataRow
        #tempDataFrame = pd.DataFrame(newDataRow, columns["target", "ids", "date", "flag", "user", "text"])
        #replyDataFrame = replyDataFrame.append(tempDataFame, ignore_index = True)
        #replyDataFrame = replyDataFrame.append(pd.DataFrame(newDataRow, columns = ["target", "ids", "date", "flag", "user", "text"]),ignore_index = True)
        #break
    #print("{} Finale Dataframe \n".format(iterator) + str(replyDataFrame.head()))
    iterator += 1 


In [ ]:
#print(dir(replyDataFrame))
print(len(replyDataFrame))
print(replyDataFrame.shape)
print(replyDataFrame.head())

In [ ]:
#print(replyDataFrame.query("date < '2022-01-01'").head())
#print(replyDataFrame.query("date < '2022-01-01'").count())
#print(replyDataFrame.query("text"[0:18] == "RT @Bundeskanzler:").head())
print("----")
#mask = replyDataFrame["text"].str.contains("RT @Bundeskanzler:").head()
#print(replyDataFrame[mask].head())
#print(replyDataFrame[replyDataFrame["text"].str.contains("RT @Bundeskanzler:")].head())
#print(replyDataFrame[replyDataFrame["text"].str.contains("RT @Bundeskanzler:")].count())
print(replyDataFrame[replyDataFrame["text"].str.contains("RT @Bundeskanzler:")].sort_values("date",ascending = "true").head())
# -> Proven that first Retweet was after 2022-02-14

In [ ]:
#testing
print(replyDataFrame["text"].str.contains("RT @Bundeskanzler").head())
print(replyDataFrame["text"].str.contains("RT @Bundeskanzler").count())

In [ ]:
#cleansedTweetDF = replyDataFrame[replyDataFrame["text"].str.contains("RT @Bundeskanzler:") or replyDataFrame["text"].str.contains("!RT @") ]
print("Size of dataframe :\n"+ str(replyDataFrame.count())+"\n----")
maskBundesKanz = replyDataFrame["text"].str.contains("RT @Bundeskanzler:")
maskNoRT = replyDataFrame["text"].str.contains("RT @")==False
print("Size of dataframe with RT @Bundeskanzler:\n" + str(replyDataFrame[maskBundesKanz].count())+"\n----")
print("Size of dataframe without RT @:\n" + str(replyDataFrame[maskNoRT].count())+"\n----")
#cleansedTweetDF contains all Tweets posted directly by Olaf before or after being Kanzler
cleansedTweetDF = replyDataFrame[maskBundesKanz | maskNoRT]
print("Size of cleansedTweetDF @:\n" + str(cleansedTweetDF.count())+"\n----")
print(cleansedTweetDF.head())
#cleansedTweetDF has "RT @Bundeskanzler:" still in the tweet. To not have any issues with ML we removed that part of the Tweet in the following parts

In [ ]:
# Removing the Text "RT @Bundeskanzler:" from the text
print("ReplyDataframe: \n" + str(replyDataFrame[maskBundesKanz].count()))
cleansedAtBundKanzDF = replyDataFrame[maskBundesKanz].copy()
#print(str(cleansedAtBundKanzDF["text"]))
cleansedAtBundKanzDF.text = cleansedAtBundKanzDF.text.str.strip("RT @Bundeskanzler: ")
print(cleansedAtBundKanzDF.head())

In [ ]:
#appending the sliced versions to the Dataframe and resulting in the expected 4119 Tweets
cleansedTweetDFShort = replyDataFrame[maskNoRT].append(cleansedAtBundKanzDF)
print(cleansedTweetDFShort.count())
print(cleansedTweetDFShort.sort_values("date",ascending = False).head())



